In [1]:
import pandas as pd
import numpy as np
from config import *
from feature_extract import *
from datetime import date

def extract_other_feature(dataset_file_path,num):
	print '第{0}个样本提取'.format(num)
	other_feature = pd.read_csv(dataset_file_path)
	other_feature.drop_duplicates(inplace=True)

	# other_feature = dataset[dataset['Date_received']!='null']

	other_feature['day_of_week'] = other_feature.Date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1 if x!='null' else 'null')
	other_feature['day_of_month'] = other_feature.Date_received.astype('str').apply(lambda x:int(x[6:8]) if x!='null' else 'null')
	other_feature['is_weekend'] = other_feature.day_of_week.apply(lambda x:1 if x in (6,7) else 0 if x!='null' else 'null')

	t1 = user_this_month_all_coupon_count(other_feature)
	t2 = user_this_day_all_coupon_count(other_feature)

	# other_feature = pd.merge(dataset,other_feature)
	other_feature = pd.merge(other_feature,t1,on='User_id')
	other_feature = pd.merge(other_feature,t2,on=['User_id','Date_received'])
	other_feature = other_feature.drop(['Discount_rate'],axis=1)
	other_feature.to_csv('data/other_feature{0}.csv'.format(num),index=None)
    
for k,path in enumerate(processed_dataset_path):
    extract_other_feature(path,k+1)

第1个样本提取
用户该月收到coupon数提取
用户该天收到coupon数提取
第2个样本提取
用户该月收到coupon数提取
用户该天收到coupon数提取
第3个样本提取
用户该月收到coupon数提取
用户该天收到coupon数提取


In [4]:
temp=pd.read_csv('data/other_feature3.csv')
temp.head()

,User_id,Merchant_id,Coupon_id,Distance,Date_received,day_of_week,day_of_month,is_weekend,user_this_month_all_coupon_count,user_this_day_all_coupon_count
0,4129537,450,9983,1,20160712,2,12,0,1,1
1,6949378,1300,3429,null,20160706,3,6,0,1,1
2,2166529,7113,6928,5,20160727,3,27,0,2,2
3,2166529,7113,1808,5,20160727,3,27,0,2,2
4,6172162,7605,6500,2,20160708,5,8,0,1,1


In [5]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112803 entries, 0 to 112802
Data columns (total 10 columns):
User_id                             112803 non-null int64
Merchant_id                         112803 non-null int64
Coupon_id                           112803 non-null int64
Distance                            112803 non-null object
Date_received                       112803 non-null int64
day_of_week                         112803 non-null int64
day_of_month                        112803 non-null int64
is_weekend                          112803 non-null int64
user_this_month_all_coupon_count    112803 non-null int64
user_this_day_all_coupon_count      112803 non-null int64
dtypes: int64(9), object(1)
memory usage: 8.6+ MB
